In [3]:
import xgboost as xgb
import pandas as pd
from tools import create_x_y

In [2]:
df = pd.read_csv('../datasets/relevant/amzn.csv')

In [6]:
VITAL_COLS = ["date_", "ticker", "close", "diffs", "bin_2", "bin_3", "bin_5"]
OTHER_COLS = [i for i in df.columns if i not in VITAL_COLS]

In [10]:
clf = xgb.XGBClassifier()
x, y = create_x_y(df, x_cols=OTHER_COLS, y_col='bin_2', lag=5)
dmatrix = xgb.DMatrix(data=x, label=y)
cv_xgb = xgb.cv(dtrain=dmatrix, params={}, nfold=5)